# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [89]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
df = df.dropna()

df.reset_index(inplace=True, drop=True)
# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [90]:
df_coded = pd.get_dummies(df, columns=["home_ownership", "verification_status", "issue_d",
                                       "pymnt_plan", "initial_list_status",
                                       "hardship_flag",  "debt_settlement_flag",
                                       "next_pymnt_d", "application_type"]
                         )
df_coded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,hardship_flag_N,debt_settlement_flag_N,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,1,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,1,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,1,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,1,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,1,1,0,1,1,0


## Split the Data into Training and Testing

In [78]:
# Create our features
X = df_coded.drop(columns = "loan_status")

# Create our target
y = df_coded["loan_status"]

In [79]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,hardship_flag_N,debt_settlement_flag_N,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,1.0,1.0,0.383161,0.616839,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.0,0.0,0.486161,0.486161,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,1.0,1.0,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,1.0,1.0,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000


In [91]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [92]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [82]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [84]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [85]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [107]:
# Resample the training data with the BalancedRandomForestClassifier
from imbalanced_ensemble.ensemble import BalancedRandomForestClassifier #Note: see: https://github.com/ZhiningLiu1998/imbalanced-ensemble/issues/6
                                                                        #Also, the imblearn.ensemble had a bug so switched to imballanced_ensemble
brf = BalancedRandomForestClassifier(n_estimators=500, random_state=1)

model = brf.fit(X_train_scaled, y_train)

y_pred_brf = model.predict(X_test_scaled)
y_pred_brf

array(['high_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [108]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred_brf)

0.8093315767633182

In [109]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_brf)

array([[   61,    26],
       [ 1412, 15706]])

In [110]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.70      0.92      0.08      0.80      0.63        87
   low_risk       1.00      0.92      0.70      0.96      0.80      0.66     17118

avg / total       0.99      0.92      0.70      0.95      0.80      0.66     17205



In [113]:
# List the features sorted in descending order by feature importance
importances = model.feature_importances_

importances_sorted = sorted(zip(model.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.06759991281414889, 'total_rec_prncp'),
 (0.06461442187231466, 'total_rec_int'),
 (0.06086339462805627, 'total_pymnt_inv'),
 (0.05811043058465312, 'last_pymnt_amnt'),
 (0.05603805723107997, 'total_pymnt'),
 (0.02814892471259863, 'int_rate'),
 (0.018746912982004, 'issue_d_Jan-2019'),
 (0.017730426710761624, 'installment'),
 (0.016821489223204908, 'dti'),
 (0.016704108442723287, 'bc_util'),
 (0.016400485508253515, 'mths_since_recent_inq'),
 (0.01639479610776567, 'max_bal_bc'),
 (0.01633850957138073, 'annual_inc'),
 (0.01614664452970594, 'out_prncp'),
 (0.016125449882608594, 'out_prncp_inv'),
 (0.015982315731848187, 'il_util'),
 (0.015309766159171172, 'mo_sin_old_rev_tl_op'),
 (0.014902859288582822, 'revol_bal'),
 (0.014106238204243745, 'tot_hi_cred_lim'),
 (0.013994092964494732, 'tot_cur_bal'),
 (0.013868661416094668, 'avg_cur_bal'),
 (0.013533470497653131, 'mo_sin_old_il_acct'),
 (0.013228877587670407, 'mths_since_rcnt_il'),
 (0.013217334525739184, 'bc_open_to_buy'),
 (0.013174871454

### Easy Ensemble Classifier

In [117]:
# Train the Classifier
from imbalanced_ensemble.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(random_state = 1)

model = eec.fit(X_train_scaled, y_train)

y_pred_eec = model.predict(X_test_scaled)
y_pred_eec

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [118]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred_eec)

0.8411190479068211

In [119]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_eec)

array([[   64,    23],
       [  914, 16204]])

In [120]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.74      0.95      0.12      0.83      0.68        87
   low_risk       1.00      0.95      0.74      0.97      0.83      0.71     17118

avg / total       0.99      0.95      0.74      0.97      0.83      0.71     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier model had the best balanced accuracy score.

   |Model| Balanced accuracy score|
   |:---:|:----------------------:|
   |Balanced Random Forest Classifier (BRFC)|0.8093315767633182|
   |Easy Ensemble Classifier (EEC)|0.8411190479068211|


2. Which model had the best recall score?

    The Easy Ensemble Classifier model had higher recall scores for both the high- and low-risk loan classifications.

   |Model|Risk|Recall score|
   |:---:|:--:|-----------:|
   |BRFC|High|0.70|
   ||Low|0.92|
   |EEC|High|0.74|
   ||Low|0.95|
   
3. Which model had the best geometric mean score?

    The Easy Ensemble Classifier model had higher geometric mean scores for both the high- and low-risk loan classifications.

   |Model|Risk|Geometric mean score|
   |:---:|:--:|-----------:|
   |BRFC|High|0.80|
   ||Low|0.80|
   |EEC|High|0.83|
   ||Low|0.83|

4. What are the top three features?

    Top three features:
    
    |Rank|Feature|Score|
    |:--:|:-----:|:---:|
    |1|total_rec_prncp|0.068|
    |2|total_rec_int|0.065|
    |3|total_pymnt_inv|0.061|
    
    